## Data Analysis Process

1. Asking Questions
2. Data Wrangling<br>
    a. Gathering Data
        - i. CSV files
        - ii. APIs
        - iii. Web Scraping
        - iv. Databases
    b. Assessing Data<br>
    c. Cleaning Data
3. Exploratory Data Analysis
4. Drawing Conclusion
5. Comunicating Results

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
patients = pd.read_csv('patients.csv')
treatments = pd.read_csv('treatments.csv')
adverse_reactions = pd.read_csv('adverse_reactions.csv')
treatments_cut = pd.read_csv('treatments_cut.csv')

In [ ]:
#view datasets

patients.head()


,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1


In [ ]:
treatments.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,veronika,jindrová,41u - 48u,-,7.63,7.20,NaN
1,elliot,richardson,-,40u - 45u,7.56,7.09,0.97
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,NaN
3,skye,gormanston,33u - 36u,-,7.97,7.62,0.35
4,alissa,montez,-,33u - 29u,7.78,7.46,0.32


In [ ]:
treatments_cut.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,jožka,resanovič,22u - 30u,-,7.56,7.22,0.34
1,inunnguaq,heilmann,57u - 67u,-,7.85,7.45,NaN
2,alwin,svensson,36u - 39u,-,7.78,7.34,NaN
3,thể,lương,-,61u - 64u,7.64,7.22,0.92
4,amanda,ribeiro,36u - 44u,-,7.85,7.47,0.38


### 1. Write a summary for your data

This is a dataset about 500 patients of which 350 patients participated in a clinical trial. None of the patients were using Novodra (a popular injectable insulin) or Auralin (the oral insulin being researched) as their primary source of insulin before. All were experiencing elevated HbA1c levels.

All 350 patients were treated with Novodra to establish a baseline HbA1c level and insulin dose. After 4 weeks, which isn’t enough time to capture all the change in HbA1c that can be attributed by the switch to Auralin or Novodra:

- 175 patients switched to Auralin for 24 weeks
- 175 patients continued using Novodra for 24 weeks

Data about patients feeling some adverse effects is also recorded.




```### 2. Write Column descriptions

#### **Table** -> `patients`:

- `patient_id`: the unique identifier for each patient in the Master Patient Index (i.e. patient database) of the pharmaceutical company that is producing Auralin
- `assigned_sex`: the assigned sex of each patient at birth (male or female)
- `given_name`: the given name (i.e. first name) of each patient
- `surname`: the surname (i.e. last name) of each patient
- `address`: the main address for each patient
- `city`: the corresponding city for the main address of each patient
- `state`: the corresponding state for the main address of each patient
- `zip_code`: the corresponding zip code for the main address of each patient
- `country`: the corresponding country for the main address of each patient (all United states for this clinical trial)
- `contact`: phone number and email information for each patient
- `birthdate`: the date of birth of each patient (month/day/year). The inclusion criteria for this clinical trial is age >= 18 (there is no maximum age because diabetes is a growing problem among the elderly population)
- `weight`: the weight of each patient in pounds (lbs)
- `height`: the height of each patient in inches (in)
- `bmi`: the Body Mass Index (BMI) of each patient. BMI is a simple calculation using a person's height and weight. The formula is BMI = kg/m2 where kg is a person's weight in kilograms and m2 is their height in metres squared. A BMI of 25.0 or more is overweight, while the healthy range is 18.5 to 24.9. The inclusion criteria for this clinical trial is 16 >= BMI >= 38.

#### **Table** -> `treatments` and `treatment_cut`:

- `given_name`: the given name of each patient in the Master Patient Index that took part in the clinical trial
- `surname`: the surname of each patient in the Master Patient Index that took part in the clinical trial
- `auralin`: the baseline median daily dose of insulin from the week prior to switching to Auralin (the number before the dash) and the ending median daily dose of insulin at the end of the 24 weeks of treatment measured over the 24th week of treatment (the number after the dash). Both are measured in units (shortform 'u'), which is the international unit of measurement and the standard measurement for insulin.
- `novodra`: same as above, except for patients that continued treatment with Novodra
- `hba1c_start`: the patient's HbA1c level at the beginning of the first week of treatment. HbA1c stands for Hemoglobin A1c. The HbA1c test measures what the average blood sugar has been over the past three months. It is thus a powerful way to get an overall sense of how well diabetes has been controlled. Everyone with diabetes should have this test 2 to 4 times per year. Measured in %.
- `hba1c_end`: the patient's HbA1c level at the end of the last week of treatment
- `hba1c_change`: the change in the patient's HbA1c level from the start of treatment to the end, i.e., hba1c_start - hba1c_end. For Auralin to be deemed effective, it must be "noninferior" to Novodra, the current standard for insulin. This "noninferiority" is statistically defined as the upper bound of the 95% confidence interval being less than 0.4% for the difference between the mean HbA1c changes for Novodra and Auralin (i.e. Novodra minus Auralin).

#### **Table** -> `adverse_reactions`

- `given_name`: the given name of each patient in the Master Patient Index that took part in the clinical trial and had an adverse reaction (includes both patients treated Auralin and Novodra)
- `surname`: the surname of each patient in the Master Patient Index that took part in the clinical trial and had an adverse reaction (includes both patients treated Auralin and Novodra)
- `adverse_reaction`: the adverse reaction reported by the patient

### 3. Add any additional information

Additional useful information:

- Insulin resistance varies person to person, which is why both starting median daily dose and ending median daily dose are required, i.e., to calculate change in dose.
- It is important to test drugs and medical products in the people they are meant to help. People of different age, race, sex, and ethnic group must be included in clinical trials. This diversity is reflected in the patients table.

### Types of Assessment
There are 2 types of assessment styles

- `Manual` - Looking through the data manually in google sheets
- `Programmatic` - By using pandas functions such as info(), describe() or sample()

### Steps in Assessment
There are 2 steps involved in Assessment

- Discover
- Document

**Issues with the dataset**
1. Dirty Data

  Table - `Patients`

    - Patient ID 9 has mispelled name 'dsvid' instaed of David `Accuracy`
    - state column contain fullname and short forms of state names `consistancy`
    - zipcode column has entries with 4 numbers some have 5 numbers `validity`
    - data missing for 12 patients in address,city,state,country,contact `Completion`
    - incorrect datatype sex,zipcode,birthdate `validity`
    - duplicate entries by the name of John Doe `Accuracy`
    - one patient has weight 48 pounds and one patient has height 27 inches `Accuracy`

  Table - `treatments ` & `treatment_cut `
    - Name and surname is in smallcase `consistancy`
    - remove u from Auralin and Novodra column `validity`
    - remove - from auralin and novodra column treated as nan `validity`
    - hba1c_change missing values showing `completing`
    - treatments dataset - one duplicatd entry of joseph(not in treatement cut) `accuracy`
    - in hba1c_change 9 instead of 4 `accuracy`

  Table - `adverse_reaction `
   - name and surname column in smallcase `consistency`

2. Messy Data

  Table - `Patients`
   - contact column contain the email and phone number combined

  Table - `treatments ` & `treatment_cut `
   - Auralin and Novodra must be split into 2 columns on start and end doses
  - merge both table
  Table - `adverse_reaction `
   - This table should not exist independently

### Automatic Assessment

- head and tail
- sample
- info
- isnull
- duplicated
- describe

In [ ]:
#patients dataset

patients.head()

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1


In [ ]:
patients.info()


# we need to change dtypes of assigned_sex  to category, birthdate to datetime,zip_code to int

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   patient_id    503 non-null    int64  
 1   assigned_sex  503 non-null    object 
 2   given_name    503 non-null    object 
 3   surname       503 non-null    object 
 4   address       491 non-null    object 
 5   city          491 non-null    object 
 6   state         491 non-null    object 
 7   zip_code      491 non-null    float64
 8   country       491 non-null    object 
 9   contact       491 non-null    object 
 10  birthdate     503 non-null    object 
 11  weight        503 non-null    float64
 12  height        503 non-null    int64  
 13  bmi           503 non-null    float64
dtypes: float64(3), int64(2), object(9)
memory usage: 55.1+ KB


In [ ]:
#there are 12 records having NaN values

patients[patients["address"].isnull()]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
209,210,female,Lalita,Eldarkhanov,NaN,NaN,NaN,NaN,NaN,NaN,8/14/1950,143.4,62,26.2
219,220,male,Mỹ,Quynh,NaN,NaN,NaN,NaN,NaN,NaN,4/9/1978,237.8,69,35.1
230,231,female,Elisabeth,Knudsen,NaN,NaN,NaN,NaN,NaN,NaN,9/23/1976,165.9,63,29.4
234,235,female,Martina,Tománková,NaN,NaN,NaN,NaN,NaN,NaN,4/7/1936,199.5,65,33.2
242,243,male,John,O'Brian,NaN,NaN,NaN,NaN,NaN,NaN,2/25/1957,205.3,74,26.4
249,250,male,Benjamin,Mehler,NaN,NaN,NaN,NaN,NaN,NaN,10/30/1951,146.5,69,21.6
257,258,male,Jin,Kung,NaN,NaN,NaN,NaN,NaN,NaN,5/17/1995,231.7,69,34.2
264,265,female,Wafiyyah,Asfour,NaN,NaN,NaN,NaN,NaN,NaN,11/3/1989,158.6,63,28.1
269,270,female,Flavia,Fiorentino,NaN,NaN,NaN,NaN,NaN,NaN,10/9/1937,175.2,61,33.1
278,279,female,Generosa,Cabán,NaN,NaN,NaN,NaN,NaN,NaN,12/16/1962,124.3,69,18.4


In [ ]:
treatments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    280 non-null    object 
 1   surname       280 non-null    object 
 2   auralin       280 non-null    object 
 3   novodra       280 non-null    object 
 4   hba1c_start   280 non-null    float64
 5   hba1c_end     280 non-null    float64
 6   hba1c_change  171 non-null    float64
dtypes: float64(3), object(4)
memory usage: 15.4+ KB


In [ ]:
treatments_cut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    70 non-null     object 
 1   surname       70 non-null     object 
 2   auralin       70 non-null     object 
 3   novodra       70 non-null     object 
 4   hba1c_start   70 non-null     float64
 5   hba1c_end     70 non-null     float64
 6   hba1c_change  42 non-null     float64
dtypes: float64(3), object(4)
memory usage: 4.0+ KB


In [ ]:
patients[patients.duplicated(subset =["given_name","surname"])]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
229,230,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4
237,238,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4
244,245,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4
251,252,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4
277,278,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4


In [ ]:
treatments[treatments.duplicated(subset =["given_name","surname"])]

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
136,joseph,day,29u - 36u,-,7.7,7.19,NaN


In [ ]:
adverse_reactions[adverse_reactions.duplicated(subset =["given_name","surname"])]

,given_name,surname,adverse_reaction


In [ ]:
patients.describe()

,patient_id,zip_code,weight,height,bmi
count,503.000000,491.000000,503.000000,503.000000,503.000000
mean,252.000000,49084.118126,173.434990,66.634195,27.483897
std,145.347859,30265.807442,33.916741,4.411297,5.276438
min,1.000000,1002.000000,48.800000,27.000000,17.100000
25%,126.500000,21920.500000,149.300000,63.000000,23.300000
50%,252.000000,48057.000000,175.300000,67.000000,27.200000
75%,377.500000,75679.000000,199.500000,70.000000,31.750000
max,503.000000,99701.000000,255.900000,79.000000,37.700000


If we see weight column the min value is 48 pounds and max is 255 pounds and the 25% is 149 pounds its huge gap maybe its issue because lower value can affect the mean same with the height column lets check  

In [ ]:
patients[patients["weight"] == 48.80]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
210,211,female,Camilla,Zaitseva,4689 Briarhill Lane,Wooster,OH,44691.0,United States,330-202-2145CamillaZaitseva@superrito.com,11/26/1938,48.8,63,19.1


In [ ]:
patients[patients["height"] == 27]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1


In [ ]:
treatments.describe()

,hba1c_start,hba1c_end,hba1c_change
count,280.000000,280.000000,171.000000
mean,7.985929,7.589286,0.546023
std,0.568638,0.569672,0.279555
min,7.500000,7.010000,0.200000
25%,7.660000,7.270000,0.340000
50%,7.800000,7.420000,0.380000
75%,7.970000,7.570000,0.920000
max,9.950000,9.580000,0.990000


In [ ]:
treatments.sort_values("hba1c_change", na_position = "first")

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,veronika,jindrová,41u - 48u,-,7.63,7.20,NaN
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,NaN
8,saber,ménard,-,54u - 54u,8.08,7.70,NaN
9,asia,woźniak,30u - 36u,-,7.76,7.37,NaN
10,joseph,day,29u - 36u,-,7.70,7.19,NaN
...,...,...,...,...,...,...,...
49,jackson,addison,-,42u - 42u,7.99,7.51,0.98
17,gina,cain,-,36u - 36u,7.88,7.40,0.98
32,laura,ehrlichmann,-,43u - 40u,7.95,7.46,0.99
245,wu,sung,-,47u - 48u,7.61,7.12,0.99


### Data Quality Dimensions

- Completeness -> is data missing?
- Validity -> is data invalid -> negative height -> duplicate patient id
- Accuracy -> data is valid but not accurate -> weight -> 1kg
- Consistency -> both valid and accurate but written differently -> New Youk and NY

Order of severity


Completeness <- Validity <- Accuracy <- Consistency

### Data Cleaning Order

1. Quality -> Completeness
2. Tidiness
3. Quality -> Validity
4. Quality -> Accuracy
5. Quality -> Consistency

#### Steps involved in Data cleaning
- Define
- Code
- Test

`Always make sure to create a copy of your pandas dataframe before you start the cleaning process`

In [ ]:
patient_df = patients.copy()
treatments_df = treatments.copy()
adverse_reactions_df = adverse_reactions.copy()
treatments_cut_df = treatments_cut.copy()

In [ ]:
patient_df.fillna("No Data",inplace = True)

In [ ]:
patient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   patient_id    503 non-null    int64  
 1   assigned_sex  503 non-null    object 
 2   given_name    503 non-null    object 
 3   surname       503 non-null    object 
 4   address       503 non-null    object 
 5   city          503 non-null    object 
 6   state         503 non-null    object 
 7   zip_code      503 non-null    object 
 8   country       503 non-null    object 
 9   contact       503 non-null    object 
 10  birthdate     503 non-null    object 
 11  weight        503 non-null    float64
 12  height        503 non-null    int64  
 13  bmi           503 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 55.1+ KB


In [ ]:
treatments.sample()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
40,ásta,grímsdóttir,-,29u - 30u,7.62,7.16,0.96


In [ ]:
treatments_df["hba1c_change"] = treatments_df["hba1c_start"] - treatments_df["hba1c_end"]
treatments_cut_df["hba1c_change"] = treatments_cut_df["hba1c_start"] - treatments_cut_df["hba1c_end"]


In [ ]:
treatments.sample()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
266,ursula,freud,42u - 54u,-,7.75,7.46,0.29


In [ ]:
treatments_cut_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    70 non-null     object 
 1   surname       70 non-null     object 
 2   auralin       70 non-null     object 
 3   novodra       70 non-null     object 
 4   hba1c_start   70 non-null     float64
 5   hba1c_end     70 non-null     float64
 6   hba1c_change  70 non-null     float64
dtypes: float64(3), object(4)
memory usage: 4.0+ KB


In [ ]:
treatments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    280 non-null    object 
 1   surname       280 non-null    object 
 2   auralin       280 non-null    object 
 3   novodra       280 non-null    object 
 4   hba1c_start   280 non-null    float64
 5   hba1c_end     280 non-null    float64
 6   hba1c_change  280 non-null    float64
dtypes: float64(3), object(4)
memory usage: 15.4+ KB


In [ ]:
patient_df["contact"].head(50)

0              951-719-9170ZoeWellish@superrito.com
1             PamelaSHill@cuvox.de+1 (217) 569-3204
2                  402-363-6804JaeMDebord@gustr.com
3        PhanBaLiem@jourrapide.com+1 (732) 636-8246
4                   334-515-7487TimNeudorf@cuvox.de
5          386-334-5237RafaelCardosoCosta@gustr.com
6                 775-533-5933MaryBAdams@einrot.com
7              XiuxiuChang@einrot.com1 408 778 3236
8           816-265-9578DavidGustafsson@armyspy.com
9     SophieCabreraIbarra@teleworm.us1 718 795 9124
10           906-478-8949SandyGunnarsson@dayrep.com
11       Abdul-NurMummarIsa@rhyta.com1 931 207 0839
12        OmeokachieIbeamaka@einrot.com434-509-2614
13       AnenechiChidi@armyspy.com+1 (205) 417-8095
14                AsiaWozniak@rhyta.com918-712-3469
15                  276-225-1955SrenFLund@gustr.com
16           LieuThiThuTam@dayrep.com1 559 765 7836
17         RoxanneAndreyeva@armyspy.com678-829-8578
18            406-775-2696WilliamVOates@armyspy.com
19          

In [ ]:
def find_contact_details(text: str) -> tuple:
    # it the value is NaN, then return it
    if pd.isna(text):
        return np.nan

    # create the phone number pattern
    phone_number_pattern = re.compile(r"(\+[\d]{1,3}\s)?(\(?[\d]{3}\)?\s?-?[\d]{3}\s?-?[\d]{4})")
    # find the phone number from the value/text, as a result we will get a list
    phone_number  = re.findall(phone_number_pattern, text)

    # if length is 0, then the regex can't find any ph number, then define with NaN
    if len(phone_number) <= 0:
        phone_number = np.nan
    # if the country code is attached with the ph number, for that case, the first
    # element will be the country code and the 2nd element will be the actual ph
    # number. So, get that ph number
    elif len(phone_number) >= 2:
        phone_number = phone_number[1]
    # else, we will get the ph number. Grab it.
    else:
        phone_number = phone_number[0]

    # if we found the ph number (with/without country code), then remove that part from the actual value.
    # after removing the ph number, the remaining string might be the email address.
    possible_email_add = re.sub(phone_number_pattern, "", text).strip()

    # then return the ph number and the email address
    return phone_number, possible_email_add

In [ ]:
patient_df['phone'] = patient_df["contact"].apply(lambda x: find_contact_details(x)).apply(lambda x:x[0])
patient_df['email'] = patient_df["contact"].apply(lambda x: find_contact_details(x)).apply(lambda x:x[1])

In [ ]:
patient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   patient_id    503 non-null    int64  
 1   assigned_sex  503 non-null    object 
 2   given_name    503 non-null    object 
 3   surname       503 non-null    object 
 4   address       503 non-null    object 
 5   city          503 non-null    object 
 6   state         503 non-null    object 
 7   zip_code      503 non-null    object 
 8   country       503 non-null    object 
 9   contact       503 non-null    object 
 10  birthdate     503 non-null    object 
 11  weight        503 non-null    float64
 12  height        503 non-null    int64  
 13  bmi           503 non-null    float64
 14  phone         491 non-null    object 
 15  email         503 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 63.0+ KB


In [ ]:
patient_df

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,phone,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6,"(, 951-719-9170)",ZoeWellish@superrito.com
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2,"(+1 , (217) 569-3204)",PamelaSHill@cuvox.de
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8,"(, 402-363-6804)",JaeMDebord@gustr.com
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7,"(+1 , (732) 636-8246)",PhanBaLiem@jourrapide.com
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1,"(, 334-515-7487)",TimNeudorf@cuvox.de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,499,male,Mustafa,Lindström,2530 Victoria Court,Milton Mills,ME,3852.0,United States,207-477-0579MustafaLindstrom@jourrapide.com,4/10/1959,181.1,72,24.6,"(, 207-477-0579)",MustafaLindstrom@jourrapide.com
499,500,male,Ruman,Bisliev,494 Clarksburg Park Road,Sedona,AZ,86341.0,United States,928-284-4492RumanBisliev@gustr.com,3/26/1948,239.6,70,34.4,"(, 928-284-4492)",RumanBisliev@gustr.com
500,501,female,Jinke,de Keizer,649 Nutter Street,Overland Park,MO,64110.0,United States,816-223-6007JinkedeKeizer@teleworm.us,1/13/1971,171.2,67,26.8,"(, 816-223-6007)",JinkedeKeizer@teleworm.us
501,502,female,Chidalu,Onyekaozulu,3652 Boone Crockett Lane,Seattle,WA,98109.0,United States,ChidaluOnyekaozulu@jourrapide.com1 360 443 2060,2/13/1952,176.9,67,27.7,"(, 360 443 2060)",ChidaluOnyekaozulu@jourrapide.com1


In [ ]:
patient_df.drop(columns = "contact",inplace = True)

In [ ]:
patient_df.head(3)

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,phone,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,7/10/1976,121.7,66,19.6,"(, 951-719-9170)",ZoeWellish@superrito.com
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,4/3/1967,118.8,66,19.2,"(+1 , (217) 569-3204)",PamelaSHill@cuvox.de
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,2/19/1980,177.8,71,24.8,"(, 402-363-6804)",JaeMDebord@gustr.com


In [ ]:
treatment_df = pd.concat([treatments_df,treatments_cut_df])

In [ ]:
treatment_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 0 to 69
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    350 non-null    object 
 1   surname       350 non-null    object 
 2   auralin       350 non-null    object 
 3   novodra       350 non-null    object 
 4   hba1c_start   350 non-null    float64
 5   hba1c_end     350 non-null    float64
 6   hba1c_change  350 non-null    float64
dtypes: float64(3), object(4)
memory usage: 21.9+ KB


In [ ]:
treatment_df =treatment_df.melt(id_vars = ["given_name","surname","hba1c_start","hba1c_end","hba1c_change"],var_name="Type",value_name = "Dosage")
treatment_df

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,Type,Dosage
0,veronika,jindrová,7.63,7.20,0.43,auralin,41u - 48u
1,elliot,richardson,7.56,7.09,0.47,auralin,-
2,yukitaka,takenaka,7.68,7.25,0.43,auralin,-
3,skye,gormanston,7.97,7.62,0.35,auralin,33u - 36u
4,alissa,montez,7.78,7.46,0.32,auralin,-
...,...,...,...,...,...,...,...
695,rovzan,kishiev,7.75,7.41,0.34,novodra,-
696,jakob,jakobsen,7.96,7.51,0.45,novodra,28u - 26u
697,bernd,schneider,7.74,7.44,0.30,novodra,-
698,berta,napolitani,7.68,7.21,0.47,novodra,42u - 44u


In [ ]:
treatment_df = treatment_df[treatment_df['Dosage'] != '-']

In [ ]:
treatment_df

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,Type,Dosage
0,veronika,jindrová,7.63,7.20,0.43,auralin,41u - 48u
3,skye,gormanston,7.97,7.62,0.35,auralin,33u - 36u
6,sophia,haugen,7.65,7.27,0.38,auralin,37u - 42u
7,eddie,archer,7.89,7.55,0.34,auralin,31u - 38u
9,asia,woźniak,7.76,7.37,0.39,auralin,30u - 36u
...,...,...,...,...,...,...,...
688,christopher,woodward,7.51,7.06,0.45,novodra,55u - 51u
690,maret,sultygov,7.67,7.30,0.37,novodra,26u - 23u
694,lixue,hsueh,9.21,8.80,0.41,novodra,22u - 23u
696,jakob,jakobsen,7.96,7.51,0.45,novodra,28u - 26u


In [ ]:
treatment_df["Start_dose"]= treatment_df["Dosage"].str.split("-").str.get(0)
treatment_df["end_dose"]= treatment_df["Dosage"].str.split("-").str.get(1)

<ipython-input-40-f36643a2b94d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treatment_df["Start_dose"]= treatment_df["Dosage"].str.split("-").str.get(0)
<ipython-input-40-f36643a2b94d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treatment_df["end_dose"]= treatment_df["Dosage"].str.split("-").str.get(1)


In [ ]:
treatment_df

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,Type,Dosage,Start_dose,end_dose
0,veronika,jindrová,7.63,7.20,0.43,auralin,41u - 48u,41u,48u
3,skye,gormanston,7.97,7.62,0.35,auralin,33u - 36u,33u,36u
6,sophia,haugen,7.65,7.27,0.38,auralin,37u - 42u,37u,42u
7,eddie,archer,7.89,7.55,0.34,auralin,31u - 38u,31u,38u
9,asia,woźniak,7.76,7.37,0.39,auralin,30u - 36u,30u,36u
...,...,...,...,...,...,...,...,...,...
688,christopher,woodward,7.51,7.06,0.45,novodra,55u - 51u,55u,51u
690,maret,sultygov,7.67,7.30,0.37,novodra,26u - 23u,26u,23u
694,lixue,hsueh,9.21,8.80,0.41,novodra,22u - 23u,22u,23u
696,jakob,jakobsen,7.96,7.51,0.45,novodra,28u - 26u,28u,26u


In [ ]:
treatment_df

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,Type,Dosage,Start_dose,end_dose
0,veronika,jindrová,7.63,7.20,0.43,auralin,41u - 48u,41u,48u
3,skye,gormanston,7.97,7.62,0.35,auralin,33u - 36u,33u,36u
6,sophia,haugen,7.65,7.27,0.38,auralin,37u - 42u,37u,42u
7,eddie,archer,7.89,7.55,0.34,auralin,31u - 38u,31u,38u
9,asia,woźniak,7.76,7.37,0.39,auralin,30u - 36u,30u,36u
...,...,...,...,...,...,...,...,...,...
688,christopher,woodward,7.51,7.06,0.45,novodra,55u - 51u,55u,51u
690,maret,sultygov,7.67,7.30,0.37,novodra,26u - 23u,26u,23u
694,lixue,hsueh,9.21,8.80,0.41,novodra,22u - 23u,22u,23u
696,jakob,jakobsen,7.96,7.51,0.45,novodra,28u - 26u,28u,26u


In [ ]:
treatment_df['Start_dose'] = treatment_df['Start_dose'].str.replace('u','')

treatment_df['end_dose'] = treatment_df['end_dose'].str.replace('u','')

<ipython-input-43-27567ce5be07>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treatment_df['Start_dose'] = treatment_df['Start_dose'].str.replace('u','')
<ipython-input-43-27567ce5be07>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treatment_df['end_dose'] = treatment_df['end_dose'].str.replace('u','')


In [ ]:
print(treatment_df.columns)


Index(['given_name', 'surname', 'hba1c_start', 'hba1c_end', 'hba1c_change',
       'Type', 'Dosage', 'Start_dose', 'end_dose'],
      dtype='object')


In [ ]:
treatment_df

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,Type,Dosage,Start_dose,end_dose
0,veronika,jindrová,7.63,7.20,0.43,auralin,41u - 48u,41,48
3,skye,gormanston,7.97,7.62,0.35,auralin,33u - 36u,33,36
6,sophia,haugen,7.65,7.27,0.38,auralin,37u - 42u,37,42
7,eddie,archer,7.89,7.55,0.34,auralin,31u - 38u,31,38
9,asia,woźniak,7.76,7.37,0.39,auralin,30u - 36u,30,36
...,...,...,...,...,...,...,...,...,...
688,christopher,woodward,7.51,7.06,0.45,novodra,55u - 51u,55,51
690,maret,sultygov,7.67,7.30,0.37,novodra,26u - 23u,26,23
694,lixue,hsueh,9.21,8.80,0.41,novodra,22u - 23u,22,23
696,jakob,jakobsen,7.96,7.51,0.45,novodra,28u - 26u,28,26


In [ ]:
treatment_df.drop(columns = "Dosage",inplace =True)

<ipython-input-46-f2c2b8e33b31>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treatment_df.drop(columns = "Dosage",inplace =True)


In [ ]:
treatment_df

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,Type,Start_dose,end_dose
0,veronika,jindrová,7.63,7.20,0.43,auralin,41,48
3,skye,gormanston,7.97,7.62,0.35,auralin,33,36
6,sophia,haugen,7.65,7.27,0.38,auralin,37,42
7,eddie,archer,7.89,7.55,0.34,auralin,31,38
9,asia,woźniak,7.76,7.37,0.39,auralin,30,36
...,...,...,...,...,...,...,...,...
688,christopher,woodward,7.51,7.06,0.45,novodra,55,51
690,maret,sultygov,7.67,7.30,0.37,novodra,26,23
694,lixue,hsueh,9.21,8.80,0.41,novodra,22,23
696,jakob,jakobsen,7.96,7.51,0.45,novodra,28,26


In [ ]:
treatment_df["Start_dose"] = treatment_df["Start_dose"].astype("int")
treatment_df["end_dose"] = treatment_df["end_dose"].astype("int")


<ipython-input-48-67479b5b5f6f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treatment_df["Start_dose"] = treatment_df["Start_dose"].astype("int")
<ipython-input-48-67479b5b5f6f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treatment_df["end_dose"] = treatment_df["end_dose"].astype("int")


In [ ]:
treatment_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 0 to 698
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    350 non-null    object 
 1   surname       350 non-null    object 
 2   hba1c_start   350 non-null    float64
 3   hba1c_end     350 non-null    float64
 4   hba1c_change  350 non-null    float64
 5   Type          350 non-null    object 
 6   Start_dose    350 non-null    int64  
 7   end_dose      350 non-null    int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 24.6+ KB


In [ ]:
treatment_df = treatment_df.merge(adverse_reactions_df,how = "left",on = ["given_name","surname"])

In [ ]:
treatment_df.sample(5)

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,Type,Start_dose,end_dose,adverse_reaction
46,idalia,moore,8.34,7.90,0.44,auralin,25,31,hypoglycemia
280,katrine,lynge,7.60,7.13,0.47,novodra,43,42,NaN
238,sabr,amari,7.94,7.57,0.37,novodra,32,27,NaN
171,žarka,rap,7.54,7.15,0.39,auralin,35,48,NaN
222,sousuke,nakahara,7.91,7.49,0.42,novodra,37,35,NaN


In [ ]:
patient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   patient_id    503 non-null    int64  
 1   assigned_sex  503 non-null    object 
 2   given_name    503 non-null    object 
 3   surname       503 non-null    object 
 4   address       503 non-null    object 
 5   city          503 non-null    object 
 6   state         503 non-null    object 
 7   zip_code      503 non-null    object 
 8   country       503 non-null    object 
 9   birthdate     503 non-null    object 
 10  weight        503 non-null    float64
 11  height        503 non-null    int64  
 12  bmi           503 non-null    float64
 13  phone         491 non-null    object 
 14  email         503 non-null    object 
dtypes: float64(2), int64(2), object(11)
memory usage: 59.1+ KB


In [ ]:
patient_df.sample(3)

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,phone,email
104,105,female,Yasmin,Silva,108 Griffin Street,Phoenix,AZ,85012.0,United States,7/24/1927,200.6,62,36.7,"(, 602 265 7408)",YasminAlmeidaSilva@einrot.com1
120,121,female,Nicoline,Østergaard,2836 Boring Lane,San Francisco,CA,94108.0,United States,12/14/1926,124.5,63,22.1,"(, 415-676-8818)",NicolinePstergaard@superrito.com
478,479,female,Edelma,Villalpando,312 Jim Rosa Lane,San Jose,CA,95134.0,United States,6/24/1977,109.6,63,19.4,"(+1 , (415) 755-6435)",EdelmaVillalpandoSantillan@teleworm.us


In [ ]:
patient_df[patient_df['zip_code'] == 'No Data']

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,phone,email
209,210,female,Lalita,Eldarkhanov,No Data,No Data,No Data,No Data,No Data,8/14/1950,143.4,62,26.2,NaN,No Data
219,220,male,Mỹ,Quynh,No Data,No Data,No Data,No Data,No Data,4/9/1978,237.8,69,35.1,NaN,No Data
230,231,female,Elisabeth,Knudsen,No Data,No Data,No Data,No Data,No Data,9/23/1976,165.9,63,29.4,NaN,No Data
234,235,female,Martina,Tománková,No Data,No Data,No Data,No Data,No Data,4/7/1936,199.5,65,33.2,NaN,No Data
242,243,male,John,O'Brian,No Data,No Data,No Data,No Data,No Data,2/25/1957,205.3,74,26.4,NaN,No Data
249,250,male,Benjamin,Mehler,No Data,No Data,No Data,No Data,No Data,10/30/1951,146.5,69,21.6,NaN,No Data
257,258,male,Jin,Kung,No Data,No Data,No Data,No Data,No Data,5/17/1995,231.7,69,34.2,NaN,No Data
264,265,female,Wafiyyah,Asfour,No Data,No Data,No Data,No Data,No Data,11/3/1989,158.6,63,28.1,NaN,No Data
269,270,female,Flavia,Fiorentino,No Data,No Data,No Data,No Data,No Data,10/9/1937,175.2,61,33.1,NaN,No Data
278,279,female,Generosa,Cabán,No Data,No Data,No Data,No Data,No Data,12/16/1962,124.3,69,18.4,NaN,No Data


In [ ]:
patient_df['zip_code'] = patient_df['zip_code'].replace('No Data', 0)


In [ ]:
#incorrect datatype sex,zipcode,birthdate `validity`
patient_df['assigned_sex'] = patient_df['assigned_sex'].astype("category")
patient_df['zip_code'] = patient_df['zip_code'].astype("int")
patient_df['birthdate'] = pd.to_datetime(patient_df['birthdate'])

In [ ]:
#zipcode column has entries with 4 numbers some have 5 numbers `validity`
patient_df['zip_code'] = patient_df['zip_code'].astype(str).str.zfill(5)


In [ ]:
patient_df['zip_code']

0      92390
1      61812
2      68467
3      07095
4      36303
       ...  
498    03852
499    86341
500    64110
501    98109
502    68324
Name: zip_code, Length: 503, dtype: object

In [ ]:
patient_df.sample(3)

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,phone,email
148,149,female,Riëtte,Scheltens,4473 Church Street,Brooklyn,NY,11227,United States,1940-09-30,198.4,67,31.1,"(, 718-418-0766)",RietteScheltens@dayrep.com
19,20,male,Zak,Kelly,994 Hill Croft Farm Road,Oroville,California,95966,United States,1988-12-13,208.8,70,30.0,"(, 530 532 8397)",ZakKelly@rhyta.com1
319,320,female,Wilma,Olofsson,2235 Catherine Drive,Fargo,ND,58102,United States,1985-11-17,154.9,69,22.9,"(, 701-235-3404)",WilmaOlofsson@teleworm.us


In [ ]:
patient_df[patient_df.duplicated(subset =["given_name","surname"])]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,phone,email
229,230,male,John,Doe,123 Main Street,New York,NY,12345,United States,1975-01-01,180.0,72,24.4,"(, 1234567890)",johndoe@email.com
237,238,male,John,Doe,123 Main Street,New York,NY,12345,United States,1975-01-01,180.0,72,24.4,"(, 1234567890)",johndoe@email.com
244,245,male,John,Doe,123 Main Street,New York,NY,12345,United States,1975-01-01,180.0,72,24.4,"(, 1234567890)",johndoe@email.com
251,252,male,John,Doe,123 Main Street,New York,NY,12345,United States,1975-01-01,180.0,72,24.4,"(, 1234567890)",johndoe@email.com
277,278,male,John,Doe,123 Main Street,New York,NY,12345,United States,1975-01-01,180.0,72,24.4,"(, 1234567890)",johndoe@email.com


In [ ]:
patient_df.drop_duplicates(subset=["given_name", "surname"], keep='first', inplace=True)


In [ ]:
patient_df[(patient_df["given_name"] == 'John')& (patient_df["surname"] == 'Doe')]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,phone,email
215,216,male,John,Doe,123 Main Street,New York,NY,12345,United States,1975-01-01,180.0,72,24.4,"(, 1234567890)",johndoe@email.com


In [ ]:
patient_df.shape

(498, 15)

In [ ]:
patient_df.describe()

,patient_id,weight,height,bmi
count,498.000000,498.000000,498.000000,498.000000
mean,252.034137,173.369076,66.580321,27.514859
std,146.067474,34.080497,4.400313,5.293793
min,1.000000,48.800000,27.000000,17.100000
25%,125.250000,148.825000,63.000000,23.225000
50%,253.500000,174.450000,67.000000,27.250000
75%,378.750000,199.725000,69.000000,31.800000
max,503.000000,255.900000,79.000000,37.700000


In [ ]:
#Q1_weight = patient_df["weight"].quantile(0.25)
#Q3_weight = patient_df["weight"].quantile(0.75)
#IQR  = Q3_weight - Q1_weight


Q1_weight = patient_df['weight'].quantile(0.25)
Q3_weight = patient_df['weight'].quantile(0.75)
IQR_weight = Q3_weight - Q1_weight

Q1_height = patient_df['height'].quantile(0.25)
Q3_height = patient_df['height'].quantile(0.75)
IQR_height = Q3_height - Q1_height


In [ ]:
lower_bound_weight = Q1_weight - 1.5 * IQR_weight
upper_bound_weight = Q3_weight + 1.5 * IQR_weight

lower_bound_height = Q1_height - 1.5 * IQR_height
upper_bound_height = Q3_height + 1.5 * IQR_height


In [ ]:
#Q1_height = patient_df["height"].quantile(0.25)
#Q3_height = patient_df["height"].quantile(0.75)
#IQR  = Q3_height - Q1_height

In [ ]:
#l_fence = Q1_height - 1.5*IQR
#u_fence =  Q3_height + 1.5*IQR

In [ ]:
patient_df = patient_df[(patient_df['weight'] >= lower_bound_weight) & (patient_df['weight'] <= upper_bound_weight) & (patient_df['height'] >= lower_bound_height) & (patient_df['height'] <= upper_bound_height)]


In [ ]:
patient_df.describe()

,patient_id,weight,height,bmi
count,495.000000,495.000000,495.000000,495.000000
mean,252.278788,173.497980,66.642424,27.541414
std,145.882971,33.657654,3.995876,5.293779
min,1.000000,102.100000,59.000000,17.100000
25%,125.500000,148.950000,63.000000,23.250000
50%,254.000000,174.200000,67.000000,27.400000
75%,378.500000,199.650000,69.000000,31.850000
max,503.000000,255.900000,78.000000,37.700000


In [ ]:
treatment_df.drop_duplicates(subset=["given_name", "surname"], keep='first', inplace=True)


In [ ]:
treatment_df[treatment_df.duplicated(subset =["given_name","surname"])]

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,Type,Start_dose,end_dose,adverse_reaction


In [ ]:
patient_df["state"].value_counts()

California    36
TX            32
New York      25
CA            24
MA            22
PA            18
NY            17
GA            15
Illinois      14
Florida       13
MI            13
OH            13
OK            13
LA            13
NJ            12
No Data       12
VA            11
MS            10
WI            10
IL            10
IN             9
MN             9
FL             9
TN             9
AL             8
NC             8
KY             8
WA             8
MO             7
ID             6
NV             6
KS             6
SC             5
IA             5
CT             5
ME             4
CO             4
Nebraska       4
RI             4
AR             4
AZ             4
ND             3
MD             3
DE             3
WV             3
OR             3
SD             3
MT             2
VT             2
DC             2
NE             2
AK             1
WY             1
NH             1
NM             1
Name: state, dtype: int64

In [ ]:
state_mapping = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
    'DC' : 'District of Columbia'
}

patient_df['state'] = patient_df['state'].replace(state_mapping)


<ipython-input-74-746a996fb7d8>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_df['state'] = patient_df['state'].replace(state_mapping)


In [ ]:
patient_df["state"].value_counts()

California              60
New York                42
Texas                   32
Illinois                24
Florida                 22
Massachusetts           22
Pennsylvania            18
Georgia                 15
Oklahoma                13
Louisiana               13
Ohio                    13
Michigan                13
No Data                 12
New Jersey              12
Virginia                11
Mississippi             10
Wisconsin               10
Minnesota                9
Tennessee                9
Indiana                  9
Alabama                  8
Washington               8
North Carolina           8
Kentucky                 8
Missouri                 7
Kansas                   6
Nebraska                 6
Idaho                    6
Nevada                   6
Iowa                     5
Connecticut              5
South Carolina           5
Rhode Island             4
Arkansas                 4
Arizona                  4
Colorado                 4
Maine                    4
N

In [ ]:
treatment_df['given_name'] = treatment_df['given_name'].str.capitalize()
treatment_df['surname'] = treatment_df['surname'].str.capitalize()


In [ ]:
treatment_df.head()

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,Type,Start_dose,end_dose,adverse_reaction
0,Veronika,Jindrová,7.63,7.20,0.43,auralin,41,48,NaN
1,Skye,Gormanston,7.97,7.62,0.35,auralin,33,36,NaN
2,Sophia,Haugen,7.65,7.27,0.38,auralin,37,42,NaN
3,Eddie,Archer,7.89,7.55,0.34,auralin,31,38,NaN
4,Asia,Woźniak,7.76,7.37,0.39,auralin,30,36,NaN


Exploratory Analysis

In [ ]:
treatment_df.head(3)

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,Type,Start_dose,end_dose,adverse_reaction
0,Veronika,Jindrová,7.63,7.20,0.43,auralin,41,48,NaN
1,Skye,Gormanston,7.97,7.62,0.35,auralin,33,36,NaN
2,Sophia,Haugen,7.65,7.27,0.38,auralin,37,42,NaN


In [ ]:
patient_df.head(3)

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,phone,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390,United States,1976-07-10,121.7,66,19.6,"(, 951-719-9170)",ZoeWellish@superrito.com
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812,United States,1967-04-03,118.8,66,19.2,"(+1 , (217) 569-3204)",PamelaSHill@cuvox.de
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467,United States,1980-02-19,177.8,71,24.8,"(, 402-363-6804)",JaeMDebord@gustr.com


In [ ]:
#how many of male and female patient are involved in test
patient_df["assigned_sex"].value_counts()

female    249
male      246
Name: assigned_sex, dtype: int64

In [ ]:
pd.crosstab(patient_df["assigned_sex"],treatment_df["Type"])

Type,auralin,novodra
assigned_sex,,
female,79,98
male,94,71


-

In [ ]:
pd.crosstab(patient_df["assigned_sex"],treatment_df["Type"],normalize = 'columns')*100

Type,auralin,novodra
assigned_sex,,
female,45.66474,57.988166
male,54.33526,42.011834
